In [1]:
import glob, os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.mask
import rasterio.plot
import matplotlib.pyplot as plt
from rasterstats import zonal_stats
from pathlib import Path

#pd.set_option('display.max_rows', None)

print('Successfully import of all libraries !')

Successfully import of all libraries !


# Input / Output

# Input

In [6]:
parcels_vector = f'/export/projects/FAO-EOStat-Senegal/Output_Nico/gps_with_data_with_buffer.shp'

ndvi_path = f'/export/projects/FAO-EOStat-Senegal/Sen4Stat/EO_DATA/SEN/28PCA_06-01_10-31/S2_preprocessing/Biophysical_Indicators/NDVI/'

## Output

# Read vector file with `geopandas`

In [7]:
gdf = gpd.read_file(parcels_vector).set_index('gid')
gdf

,filename,id,area,collect,Informatio,Informat_1,Informat_2,crop_code,crop_name,geometry
gid,,,,,,,,,,
0,Piste_6220102-2-4-1-1.gpx,62201022411,5172.41,GPS,1.0,1.0,5266.50,1.0,Arachide,"POLYGON ((429457.856 1516229.002, 429465.169 1..."
1,Piste_6220102-2-4-1-2.gpx,62201022412,12162.62,GPS,1.0,11.0,16867.89,11.0,Mil,"POLYGON ((429828.180 1516150.340, 429839.170 1..."
2,Piste_6220102-2-4-1-3.gpx,62201022413,6209.42,GPS,1.0,11.0,6282.66,11.0,Mil,"POLYGON ((429883.899 1516457.550, 429884.205 1..."
3,Piste_6220102-2-4-1-4.gpx,62201022414,2772.11,GPS,1.0,9.0,3493.89,9.0,Maïs,"POLYGON ((429986.736 1516281.999, 429987.433 1..."
4,Piste_6220102-2-16-4.gpx,62201022164,1860.82,GPS,NaN,NaN,NaN,NaN,None,"POLYGON ((429722.291 1515343.246, 429740.288 1..."
...,...,...,...,...,...,...,...,...,...,...
316,Piste_6220206-7-7-2-2.gpx,62202067722,4393.56,GPS,NaN,NaN,NaN,NaN,None,"POLYGON ((423666.487 1531218.677, 423664.305 1..."
317,Piste_6220206-7-7-2-3.gpx,62202067723,6802.07,GPS,NaN,NaN,NaN,NaN,None,"POLYGON ((425034.700 1531408.852, 425036.310 1..."
318,Piste_6220206-7-7-2-4.gpx,62202067724,6438.62,GPS,NaN,NaN,NaN,NaN,None,"POLYGON ((425355.728 1531218.177, 425355.744 1..."


In [9]:
df_list = []
for im in glob.glob(f'{ndvi_path}*.tif'):
    print(im)
    date = os.path.basename(im)[6:6+8]
    df = pd.DataFrame(zonal_stats(gdf, im, stats='std', nodata=-10000))
    df = df.rename(columns={'std': date})
    df_list.append(df)

dfs = pd.concat(df_list, axis=1)
dfs

/export/projects/FAO-EOStat-Senegal/Sen4Stat/EO_DATA/SEN/28PCA_06-01_10-31/S2_preprocessing/Biophysical_Indicators/NDVI/28PCA_20210601_NDVI.tif
/export/projects/FAO-EOStat-Senegal/Sen4Stat/EO_DATA/SEN/28PCA_06-01_10-31/S2_preprocessing/Biophysical_Indicators/NDVI/28PCA_20210611_NDVI.tif
/export/projects/FAO-EOStat-Senegal/Sen4Stat/EO_DATA/SEN/28PCA_06-01_10-31/S2_preprocessing/Biophysical_Indicators/NDVI/28PCA_20210621_NDVI.tif
/export/projects/FAO-EOStat-Senegal/Sen4Stat/EO_DATA/SEN/28PCA_06-01_10-31/S2_preprocessing/Biophysical_Indicators/NDVI/28PCA_20210701_NDVI.tif
/export/projects/FAO-EOStat-Senegal/Sen4Stat/EO_DATA/SEN/28PCA_06-01_10-31/S2_preprocessing/Biophysical_Indicators/NDVI/28PCA_20210711_NDVI.tif
/export/projects/FAO-EOStat-Senegal/Sen4Stat/EO_DATA/SEN/28PCA_06-01_10-31/S2_preprocessing/Biophysical_Indicators/NDVI/28PCA_20210721_NDVI.tif
/export/projects/FAO-EOStat-Senegal/Sen4Stat/EO_DATA/SEN/28PCA_06-01_10-31/S2_preprocessing/Biophysical_Indicators/NDVI/28PCA_20210731_N

,20210601,20210611,20210621,20210701,20210711,20210721,20210731,20210810,20210820,20210830,20210909,20210919,20210929,20211009,20211019
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
dict_list = []

for i, row in gdf.iterrows():

    name  = row[adm_name_field]
    
    dict_freq = zonal_stats(row.geometry,
                            crop_map,
                            categorical=True,
                            category_map=dict_rename,
                            nodata=-999)[0]

    dict_freq['name'] = name

    dict_list.append(dict_freq)

print('Display the 2 first dictionaries of the list')
dict_list[0:2]